In [33]:
import os as os
import pandas as pd
import numpy as np
    
path = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff/references/'
fname = '20180727_BioMart_ENSv78_UTRs.tsv.gz'
fpath = os.path.join(path, fname)

table = pd.read_csv(fpath, delimiter='\t', header=0)
table.columns = ['name', 'transcript', '5p_utr_start', '5p_utr_end',
                 '3p_utr_start', '3p_utr_end', 'transcript_count']

table.fillna(0, inplace=True)

table['5p_utr_length'] = table['5p_utr_end'] - table['5p_utr_start']
table['3p_utr_length'] = table['3p_utr_end'] - table['3p_utr_start']

selector = np.logical_and(table['5p_utr_length'] == 0, table['3p_utr_length'] == 0)
table = table.loc[~selector, :].copy()

int_columns = ['5p_utr_start', '5p_utr_end', '3p_utr_start', '3p_utr_end',
               'transcript_count', '5p_utr_length', '3p_utr_length']

table.loc[:, int_columns] = table[int_columns].astype(np.int32)
table.drop_duplicates(inplace=True)

out_file = os.path.join(path, '20180727_BioMart_ENSv78_UTRs.h5')

with pd.HDFStore(out_file, 'w') as hdf:
    hdf.put('Ensembl_v78/utr', table, format='fixed')
    